In [ ]:
################ Faulty Code ################

from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

In [3]:
##Reading PDF
loader = PyPDFDirectoryLoader("./us_census")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
final_documents = text_splitter.split_documents(documents)

final_documents[0]

Document(metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 18.2 (Windows)', 'creationdate': '2023-09-09T07:52:17-04:00', 'author': 'U.S. Census Bureau', 'keywords': 'acsbr-015', 'moddate': '2023-09-12T14:44:47+01:00', 'title': 'Health Insurance Coverage Status and Type by Geography: 2021 and 2022', 'trapped': '/false', 'source': 'us_census\\acsbr-015.pdf', 'total_pages': 18, 'page': 0, 'page_label': '1'}, page_content='Health Insurance Coverage Status and Type \nby Geography: 2021 and 2022\nAmerican Community Survey Briefs\nACSBR-015\nIssued September 2023\nDouglas Conway and Breauna Branch\nINTRODUCTION\nDemographic shifts as well as economic and govern-\nment policy changes can affect people’s access to \nhealth coverage. For example, between 2021 and 2022, \nthe labor market continued to improve, which may \nhave affected private coverage in the United States \nduring that time.\n1 Public policy changes included \nthe renewal of the Public Health Emergency, 

In [4]:
## Embedding Using Huggingface
huggingface_embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",
    model_kwargs={'device':"cpu"},
    encode_kwargs={'normalize_embeddings': True}
)

C:\Users\hp\AppData\Local\Temp\ipykernel_29088\1218065419.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  huggingface_embeddings = HuggingFaceEmbeddings(


In [5]:
import  numpy as np
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)))
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)).shape)

[-8.30601528e-02 -1.45066706e-02 -2.10276805e-02  2.72682551e-02
  4.53647189e-02  5.28341383e-02 -2.53759213e-02  3.61304022e-02
 -9.08312425e-02 -2.77017597e-02  7.97397718e-02  6.42474815e-02
 -3.54004018e-02 -4.04245928e-02 -1.13772415e-02  4.45296019e-02
 -3.88542423e-03 -3.79060814e-03 -4.54510413e-02  2.67046951e-02
 -2.05681641e-02  2.87432466e-02 -2.41201185e-02 -3.69412303e-02
  1.92781053e-02  1.06194559e-02  3.21826804e-03  2.33249855e-03
 -4.29321602e-02 -1.64999202e-01  2.77008908e-03  2.68276669e-02
 -4.12894525e-02 -1.88446585e-02  1.58918444e-02  9.22320783e-03
 -2.00687721e-02  8.16561431e-02  3.89413126e-02  5.52223213e-02
 -3.69984470e-02  1.75319053e-02 -1.28966765e-02  2.80639302e-04
 -2.51580607e-02  4.59338212e-03 -2.39579398e-02 -5.76566067e-03
  6.02950901e-03 -3.61178257e-02  3.84415649e-02 -1.75470242e-03
  5.05656376e-02  6.02408908e-02  4.52067964e-02 -4.91435081e-02
  1.82053987e-02 -1.46668749e-02 -2.53130775e-02  3.18243764e-02
  5.15598208e-02 -9.32342

In [6]:
## VectorStore Creation
vectorstore=FAISS.from_documents(final_documents[:120],huggingface_embeddings)

## Query using Similarity Search
query="WHAT IS HEALTH INSURANCE COVERAGE?"
relevant_docments=vectorstore.similarity_search(query)

print(relevant_docments[0].page_content)

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.


In [7]:
retriever=vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":3})
print(retriever)

tags=['FAISS', 'HuggingFaceEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000028AE085A3C0> search_kwargs={'k': 3}


In [8]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ['HUGGINGFACEHUB_API_TOKEN']=os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [12]:
from langchain_huggingface import HuggingFaceEndpoint
import os
from langchain_core.prompts import PromptTemplate

# Ensure the API token is loaded from environment
api_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")
if not api_token:
    raise ValueError("HUGGINGFACEHUB_API_TOKEN is not set in environment variables.")

hf = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-v0.1",
    huggingfacehub_api_token=api_token,
    temperature=0.1,
    # model_kwargs={"max_length":500}
)

question = "Who won the FIFA World Cup in the year 1994? "

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate.from_template(template)

llm_chain = prompt | hf

query = "What is the health insurance coverage?"
try:
    result = llm_chain.invoke({"question": question})
    if not result:
        print("No response received from the model.")
    else:
        print(result)
except StopIteration:
    print("StopIteration error: Model did not return a valid response.")
except Exception as e:
    print(f"An error occurred: {e}")


StopIteration error: Model did not return a valid response.


In [1]:
#Hugging Face models can be run locally through the HuggingFacePipeline class.
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

hf = HuggingFacePipeline.from_model_id(
    model_id="mistralai/Mistral-7B-v0.1",
    task="text-generation",
    pipeline_kwargs={"temperature": 0, "max_new_tokens": 300}
)

llm = hf 
llm.invoke(query)

d:\Python\Chatbot\chatbot1\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

: 

In [1]:
prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

Helpful Answers:
 """

prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])

NameError: name 'PromptTemplate' is not defined

In [ ]:
retrievalQA=RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [ ]:
query="""DIFFERENCES IN THE
UNINSURED RATE BY STATE
IN 2022"""

# Call the QA chain with our query.
result = retrievalQA.invoke({"query": query})
print(result['result'])